## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,75.02,34,1,1.99,clear sky
1,1,Mahebourg,MU,-20.4081,57.7000,70.25,94,75,9.22,light rain
2,2,Okha,RU,53.5739,142.9478,43.16,93,100,11.07,overcast clouds
3,3,Katangli,RU,51.7102,143.2326,41.79,93,88,7.31,overcast clouds
4,4,Luderitz,NaN,-26.6481,15.1594,60.69,69,0,11.77,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 90
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
34,34,Palatka,US,29.6486,-81.6376,93.02,46,100,3.47,overcast clouds
37,37,Springdale,US,36.1867,-94.1288,92.19,56,1,4.00,clear sky
46,46,Hun,LY,29.1268,15.9477,91.04,16,0,9.66,clear sky
100,100,Guthrie,US,35.8789,-97.4253,97.45,46,1,10.36,clear sky
155,155,San Antonio,US,29.4241,-98.4936,95.02,54,1,5.75,clear sky
162,162,Simplicio Mendes,BR,-7.8539,-41.9103,96.40,23,19,8.10,few clouds
171,171,Russell,US,32.3502,-85.1999,91.15,41,75,5.75,broken clouds
185,185,Lompoc,US,34.6391,-120.4579,94.21,53,1,5.75,clear sky
195,195,Fort Abbas,PK,29.1925,72.8536,94.35,34,83,7.81,broken clouds
212,212,Diffa,NE,13.3154,12.6113,96.39,24,70,11.59,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                35
City                   35
Country                35
Lat                    35
Lng                    35
Max Temp               35
Humidity               35
Cloudiness             35
Wind Speed             35
Current Description    35
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
34,Palatka,US,93.02,overcast clouds,29.6486,-81.6376,
37,Springdale,US,92.19,clear sky,36.1867,-94.1288,
46,Hun,LY,91.04,clear sky,29.1268,15.9477,
100,Guthrie,US,97.45,clear sky,35.8789,-97.4253,
155,San Antonio,US,95.02,clear sky,29.4241,-98.4936,
162,Simplicio Mendes,BR,96.40,few clouds,-7.8539,-41.9103,
171,Russell,US,91.15,broken clouds,32.3502,-85.1999,
185,Lompoc,US,94.21,clear sky,34.6391,-120.4579,
195,Fort Abbas,PK,94.35,broken clouds,29.1925,72.8536,
212,Diffa,NE,96.39,broken clouds,13.3154,12.6113,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace = True)

In [16]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
34,Palatka,US,93.02,overcast clouds,29.6486,-81.6376,Holiday Inn Express & Suites Palatka Northwest...
37,Springdale,US,92.19,clear sky,36.1867,-94.1288,Springdale Inn
46,Hun,LY,91.04,clear sky,29.1268,15.9477,Alrwasi Hotel
100,Guthrie,US,97.45,clear sky,35.8789,-97.4253,Holiday Inn Express & Suites Guthrie North Edm...
155,San Antonio,US,95.02,clear sky,29.4241,-98.4936,Drury Plaza Hotel San Antonio Riverwalk


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [26]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))